# case 1

In [1]:
from __future__ import annotations
import os
os.chdir('/home/lq/LQcode/2_project/PHMBench/PHMGA/')
print(f"Current working directory: {os.getcwd()}")
from langgraph.graph import StateGraph, END, START

from src.agents.dataset_preparer_agent import dataset_preparer_agent
from src.agents.execute_agent import execute_agent
from src.agents.inquirer_agent import inquirer_agent
from src.agents.plan_agent import plan_agent
from src.agents.reflect_agent import reflect_agent_node
from src.agents.report_agent import report_agent_node
from src.agents.shallow_ml_agent import shallow_ml_agent
from src.states.phm_states import PHMState
import yaml
import os
from src.utils import load_state, save_state



Current working directory: /home/lq/LQcode/2_project/PHMBench/PHMGA


/home/lq/.conda/envs/PA/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


config_path = "config/case_exp2.yaml"
print(f"--- Loading configuration from {config_path} ---")
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

state_save_path = config['state_save_path']
builder_cfg = config.get('builder', {})
min_depth = builder_cfg.get('min_depth', 0)
max_depth = builder_cfg.get('max_depth', float('inf'))

# --- Check for existing state ---
if os.path.exists(state_save_path):
    print(f"\n--- Found existing state file at {state_save_path}. Skipping builder workflow. ---")
    state = load_state(state_save_path)






--- Loading configuration from config/case_exp2.yaml ---

--- Found existing state file at /home/lq/LQcode/2_project/PHMBench/PHMGA/save/exp2/exp2_built_state.pkl. Skipping builder workflow. ---

--- Loading state from /home/lq/LQcode/2_project/PHMBench/PHMGA/save/exp2/exp2_built_state.pkl ---
...done.
Successfully loaded state with 40 nodes.


## 1 inquiring

In [8]:
print("\nStep 1: Inquiring - Performing similarity analysis...")
inquirer_updates = inquirer_agent(state, metrics=["cosine", "euclidean"])
state = state.copy(update=inquirer_updates)
print(f"Output: Updated state with similarity_results.")


Step 1: Inquiring - Performing similarity analysis...
Calculating similarity for 32 leaf nodes with metrics: ['cosine', 'euclidean']
Calculated cosine similarity for node rms_05_ch1
Calculated euclidean similarity for node rms_05_ch1
Calculated cosine similarity for node rms_06_ch2
Calculated euclidean similarity for node rms_06_ch2
Calculated cosine similarity for node kurtosis_01_hilbert_envelope_01_ch1
Calculated euclidean similarity for node kurtosis_01_hilbert_envelope_01_ch1
Calculated cosine similarity for node kurtosis_02_hilbert_envelope_02_ch2
Calculated euclidean similarity for node kurtosis_02_hilbert_envelope_02_ch2
Calculated cosine similarity for node spectral_kurtosis_03_fft_03_ch1
Calculated euclidean similarity for node spectral_kurtosis_03_fft_03_ch1
Calculated cosine similarity for node spectral_kurtosis_04_fft_04_ch2
Calculated euclidean similarity for node spectral_kurtosis_04_fft_04_ch2
Calculated cosine similarity for node entropy_05_ch1
Calculated euclidean si

/tmp/ipykernel_520428/2378391913.py:3: PydanticDeprecatedSince20: The `copy` method is deprecated; use `model_copy` instead. See the docstring of `BaseModel.copy` for details about how to handle `include` and `exclude`. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  state = state.copy(update=inquirer_updates)


## 2 data preparation

In [ ]:
# from src.utils import load_signal_data
# metadata_path, h5_path, ref_ids,test_ids = config['metadata_path'], config['h5_path'], config['ref_ids'], config['test_ids']
# ref_signals, ref_labels,test_metadata_df = load_signal_data(metadata_path, h5_path, ref_ids)
# test_signals, test_labels,test_metadata_df = load_signal_data(metadata_path, h5_path, test_ids)


Loading data for IDs: [47050, 47052, 47044, 47046, 47047, 47049, 47053, 47055, 47056, 47058]
Loading data for IDs: [47051, 47045, 47048, 47054, 47057]


In [3]:
# 2. Dataset Preparer Agent: Prepare datasets for ML
print("\nStep 2: Preparing - Creating datasets for ML model...")
preparer_updates = dataset_preparer_agent(state)
state = state.copy(update=preparer_updates)
print(f"Output: Updated state with datasets.")

datasets = preparer_updates['datasets']



Step 2: Preparing - Creating datasets for ML model...
Output: Updated state with datasets.


/tmp/ipykernel_520428/1127098262.py:4: PydanticDeprecatedSince20: The `copy` method is deprecated; use `model_copy` instead. See the docstring of `BaseModel.copy` for details about how to handle `include` and `exclude`. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  state = state.copy(update=preparer_updates)


## save and load dataset

In [ ]:
# save and load dataset dict
import pickle
import os
# save_dir  = '/home/lq/LQcode/2_project/PHMBench/PHMGA/save/exp2.5/'
save_dir = config.get('save_dir', 'save/exp2/')
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, 'exp2/datasets.pkl')
with open(save_path, 'wb') as f:
    pickle.dump(datasets, f)
# with open(save_path, 'rb') as f:
#     datasets = pickle.load(f)

In [5]:
config

{'name': 'exp2',
 'save_dir': '/home/lq/LQcode/2_project/PHMBench/PHMGA/save',
 'metadata_path': '/mnt/crucial/LQ/PHM-Vibench/metadata_6_11.xlsx',
 'h5_path': '/mnt/crucial/LQ/PHM-Vibench/cache.h5',
 'state_save_path': '/home/lq/LQcode/2_project/PHMBench/PHMGA/save/exp2/exp2_built_state.pkl',
 'report_path': '/home/lq/LQcode/2_project/PHMBench/PHMGA/save/exp2/exp2_final_report.md',
 'ref_ids': [47050,
  47052,
  47044,
  47046,
  47047,
  47049,
  47053,
  47055,
  47056,
  47058],
 'test_ids': [47051, 47045, 47048, 47054, 47057],
 'builder': {'min_depth': 4, 'max_depth': 8},
 'user_instruction': 'Analyze the bearing signals for potential faults. The reference set contains signals for 5 different states (health, ball, cage, inner, outer). The test set also contains signals for the same 5 states. The goal is to correctly classify each test signal by comparing it to the reference set.\n'}

## 3  train model

In [7]:
%load_ext autoreload
%autoreload 2

In [ ]:
# 3. Shallow ML Agent: Train and evaluate a simple ML model
print("\nStep 3: Training - Running shallow ML model...")
# Note: shallow_ml_agent directly takes the datasets, not the full state
ml_updates = {"ml_results": shallow_ml_agent(datasets)}

print(f"Output: Updated state with ml_results.")




Step 3: Training - Running shallow ML model...


--- Ensemble: Found 2 models with CV accuracy > 90% for ensembling.
--- High-quality models: ['fft_03_ch1', 'fft_04_ch2']
Output: Updated state with ml_results.


In [ ]:
state = state.copy(update=ml_updates)

In [13]:
result = ml_updates['ml_results']['metrics_markdown']
result

: 

: 

## 4 reporting

In [12]:
# 4. Report Agent: Generate the final report
print("\nStep 4: Reporting - Generating final analysis report...")
report_updates = report_agent_node(state)
state = state.copy(update=report_updates)
print("Output: Updated state with the final report.")


Step 4: Reporting - Generating final analysis report...
Generating final report with 40 nodes, 32 leaves, 32 similarity stats, 0 ML models, 0 ensemble metrics, issues: 0


DecodeError: Error parsing message with type 'google.ai.generativelanguage.v1beta.Part'

In [ ]:
from src.utils import generate_final_report
report = generate_final_report(state,'./src/reports/final_report.md')
print("\nFinal Report:")
print(report)


--- Workflow Finished ---


AttributeError: 'PHMState' object has no attribute 'get'

## u-map

In [10]:
from src.utils.visualization import visualize_datasets_umap

In [11]:
save_dir = '/home/lq/LQcode/2_project/PHMBench/PHMGA/save/exp2/'
visualize_datasets_umap(datasets, save_dir=save_dir)

--- Generating UMAP visualizations for 38 datasets ---
Processing: hilbert_envelope_01_ch1
  Saved visualization to /home/lq/LQcode/2_project/PHMBench/PHMGA/save/exp2/hilbert_envelope_01_ch1_umap.png
Processing: hilbert_envelope_02_ch2
  Saved visualization to /home/lq/LQcode/2_project/PHMBench/PHMGA/save/exp2/hilbert_envelope_02_ch2_umap.png
Processing: fft_03_ch1
  Saved visualization to /home/lq/LQcode/2_project/PHMBench/PHMGA/save/exp2/fft_03_ch1_umap.png
Processing: fft_04_ch2
  Saved visualization to /home/lq/LQcode/2_project/PHMBench/PHMGA/save/exp2/fft_04_ch2_umap.png
Processing: rms_05_ch1
  Saved visualization to /home/lq/LQcode/2_project/PHMBench/PHMGA/save/exp2/rms_05_ch1_umap.png
Processing: rms_06_ch2
  Saved visualization to /home/lq/LQcode/2_project/PHMBench/PHMGA/save/exp2/rms_06_ch2_umap.png
Processing: kurtosis_01_hilbert_envelope_01_ch1
  Saved visualization to /home/lq/LQcode/2_project/PHMBench/PHMGA/save/exp2/kurtosis_01_hilbert_envelope_01_ch1_umap.png
Processing

# save state

In [7]:
state_save_path = config['state_save_path']
state_save_path


'/home/lq/LQcode/2_project/PHMBench/PHMGA/save/exp2.5/exp2.5_built_state.pkl'

In [8]:

# Save the final state
print(f"\n--- Saving final state to {state_save_path} ---")
save_state(state, state_save_path)


--- Saving final state to /home/lq/LQcode/2_project/PHMBench/PHMGA/save/exp2.5/exp2.5_built_state.pkl ---

--- Saving state to /home/lq/LQcode/2_project/PHMBench/PHMGA/save/exp2.5/exp2.5_built_state.pkl ---
...done.


True

# case 2.0

# case rerun new

# new data